# Download and process sentinel 1 data

## John Brandt
## April 1, 2020

## Package imports, API import, source scripts

In [1]:
import datetime
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import scipy.sparse as sparse
import seaborn as sns
import yaml

from collections import Counter
from osgeo import ogr, osr
from random import shuffle
from scipy.sparse.linalg import splu
from sentinelhub import WmsRequest, WcsRequest, MimeType
from sentinelhub import CRS, BBox, constants, DataSource, CustomUrlParam
from skimage.transform import resize

with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%matplotlib inline
%run ../src/slope.py
%run ../src/utils.py
%run ../src/dsen2/utils/DSen2Net.py
%run ../src/utils/download_utils.py

/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [2]:
time = ('2018-12-15', '2020-01-15')
IMSIZE = 32
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

# Bounding boxes

In [4]:
def calc_bbox(plot_id, df):
    """ Calculates the corners of a bounding box from an input
        pandas dataframe as output by Collect Earth Online

        Parameters:
         plot_id (int): plot_id of associated plot
         df (pandas.DataFrame): dataframe of associated CEO survey
    
        Returns:
         bounding_box (list): [(min(x), min(y)),
                              (max(x), max_y))]
    """
    subs = df[df['PLOT_ID'] == plot_id]
    return [(min(subs['LON']), min(subs['LAT'])),
            (max(subs['LON']), max(subs['LAT']))]

def bounding_box(points, expansion = 160):
    """ Calculates the corners of a bounding box with an
        input expansion in meters from a given bounding_box
        
        Subcalls:
         calculate_epsg, convertCoords

        Parameters:
         points (list): output of calc_bbox
         expansion (float): number of meters to expand or shrink the
                            points edges to be
    
        Returns:
         bl (tuple): x, y of bottom left corner with edges of expansion meters
         tr (tuple): x, y of top right corner with edges of expansion meters
    """
    bl = list(points[0])
    tr = list(points[1])
    inproj = Proj('epsg:4326')
    outproj_code = calculate_epsg(bl)
    outproj = Proj('epsg:' + str(outproj_code))
    
    bl_utm =  transform(inproj, outproj, coord[1], coord[0])
    tr_utm =  transform(inproj, outproj, coord[1], coord[0])
    
    distance1 = tr_utm[0] - bl_utm[0]
    distance2 = tr_utm[1] - bl_utm[1]
    expansion1 = (expansion - distance1)/2
    expansion2 = (expansion - distance2)/2
    
    bl_utm = [bl_utm[0] - expansion1, bl_utm[1] - expansion2]
    tr_utm = [tr_utm[0] + expansion1, tr_utm[1] + expansion2]

    
    zone = str(outproj_code)[3:]
    direction = 'N' if coord[1] >= 0 else 'S'
    utm_epsg = "UTM_" + zone + direction
    return bl_utm, tr_utm, CRS[utm_epsg]

# Data download

In [6]:
def download_sentinel_1(bbox, epsg, time = time, 
                        layer = "SENT", year = 2019, image_format = MimeType.TIFF_d16):
    """ Downloads all 10 and 20 meter L2A bands from sentinel-hub
        for input bbox and epsg, within time range
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): UTM EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         s1 (arr): (Time, X, Y, 2) array of sentinel 1 data
         image_dates (list): number of days since time[0] for each
                              image in s1.shape[0]
    """
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer=layer,
                bbox=box,
                time=time,
                image_format = image_format,
                maxcc=1.0,
                resx='5m', resy='5m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=72),
            )
        data_filter = None
        if len(image_request.download_list) > 50:
            data_filter = [x for x in range(len(image_request.download_list)) if x % 2 == 0]
        img_bands = image_request.get_data(data_filter = data_filter)
        s1 = np.stack(img_bands)
        s1 = resize(s1, (s1.shape[0], IMSIZE*2, IMSIZE*2, s1.shape[-1]), order = 0)
        s1 = np.reshape(s1, (s1.shape[0], s1.shape[1]//2, 2, s1.shape[2] // 2, 2, s1.shape[-1]))
        s1 = np.mean(s1, (2, 4))
        s1 = s1[:, 8:24, 8:24, :]
        
        image_dates = []
        for date in image_request.get_dates():
            if date.year == year - 1:
                image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
            if date.year == year:
                image_dates.append(starting_days[(date.month-1)] + date.day)
            if date.year == year + 1:
                image_dates.append(365 + starting_days[(date.month-1)]+date.day)
        image_dates = np.array(image_dates)
        s1c = np.copy(s1)
        s1c[np.where(s1c < 1.)] = 0
        n_pix_oob = np.sum(s1c, axis = (1, 2, 3))
        to_remove = np.argwhere(n_pix_oob > (imsize*2*imsize*2)/50)
        s1 = np.delete(s1, to_remove, 0)
        image_dates = np.delete(image_dates, to_remove)
        return s1, image_dates

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Download function

In [8]:
starting_days = np.cumsum([0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30])
print(starting_days)

[  0  31  59  90 120 151 181 212 243 273 304 334]


In [9]:
super_resolve = True
year = 2019

DATA_LOCATION = '../data/ghana-test.csv'
OUTPUT_FOLDER = '../data/test-smooth-200/'

def download_plots(data_location = DATA_LOCATION, output_folder = OUTPUT_FOLDER, image_format = MimeType.TIFF_d16):
    """ Downloads sentinel-1 data for the plot IDs associated
        with an input CSV from a collect earth online survey
        
        Parameters:
         data_location (os.path)
         output_folder (os.path)
        
        Subcalls:
         calc_bbox, bounding_box
         download_sentinel_1,
         calculate_and_save_best_images
         
        Creates:
         output_folder/{plot_id}.npy
    
        Returns:
         None
    """
    print(data_location)
    df = pd.read_csv(data_location, encoding = "ISO-8859-1")
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    existing = existing + [136397663, 136792033, 136792071, 136397414, 136792213,
                          136792216, 136792229]
    to_download = [x for x in plot_ids if x not in existing]
    print("STARTING DOWNLOAD OF {} plots from {} to {}".format(len(to_download), data_location, output_folder))
    errors = []
    for i, val in enumerate(to_download):
        print("Downloading {}/{}, {}".format(i+1, len(to_download), val))
        location = calc_bbox(val, df = df)
        location, epsg = bounding_box(location, expansion = IMSIZE*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            s1, s1_dates = download_sentinel_1(location, epsg = epsg, layer = 'SENT',
                                               image_format = image_format)
            print("ASCENDING: {}".format(s1.shape))
            if s1.shape[0] == 0:
                s1, s1_dates = download_sentinel_1(location, layer = "SENT_DESC")
                print("DESCENDING: {}".format(s1.shape))
            if s1_dates.shape[0] > 0:
                s1, max_distance_s1 = calculate_and_save_best_images(s1, s1_dates)

                # Retain only iamgery every 15 days
                biweekly_dates = np.array([day for day in range(0, 360, 5)])
                to_remove = np.argwhere(biweekly_dates % 15 != 0)
                s1 = np.delete(s1, to_remove, 0)

                if max_distance_s1 <= 240:
                    np.save(output_folder + str(val), s1)
                    print("\n")
                else:
                    print("Skipping {} because there is a {} distance".format(val, max_distance))
                    print("\n")

        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)
            errors.append(img)
            #continue

In [12]:
for i in (os.listdir("../data/drylands/csv/")):
    #if "australia" in i:
    if "fao-test-2.csv" in i:
        #if any(x in i for x in ["africa-west", "cameroon", "koure", "niger"]):
            download_plots("../data/drylands/csv/" + i, "../data/drylands/s1/", image_format = MimeType.TIFF_d16)

../data/drylands/csv/fao-test-2.csv
STARTING DOWNLOAD OF 102 plots from ../data/drylands/csv/fao-test-2.csv to ../data/drylands/s1/
0 136397012
29.43742928228564 -23.714241107309793
[320.0, 320.0]
ERROR: Initial field less than 130m
(79, 65, 63, 2)
(79, 64, 64, 2)
(79, 32, 2, 32, 2, 2)
(79, 32, 32, 2)
(79, 16, 16, 2)
[512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512]
ASCENDING: (13, 16, 16, 2)
Maximum time distance: 35
